In [26]:
from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit.primitives import StatevectorSampler 
from qiskit_aer import AerSimulator
import numpy as np

# Initialize the StatevectorSampler primitive for accurate local simulation
# This object is optimized for generating counts from a noiseless statevector.
sampler = StatevectorSampler()

def create_bomb_tester_circuit(bomb_status):
    """
    Creates the Mach-Zehnder interferometer circuit with the bomb in the upper path.
    """
    # Define circuit with one qubit and one classical bit (for result 'c')
    qc = QuantumCircuit(1, 1, name=f"Bomb_Tester_{bomb_status}")

    # 1. First 50/50 Beam Splitter
    qc.h(0)
    
    # --- The "Bomb" Component (Applied to the |1> component) ---
    if bomb_status == "live":
        # Live bomb: Z gate (180-degree phase shift)
        qc.z(0)
    elif bomb_status == "dud":
        # Dud bomb: Identity gate (qc.id)
        qc.id(0)
    else:
        raise ValueError("bomb_status must be 'live' or 'dud'.")
    
    # 2. Second 50/50 Beam Splitter
    qc.h(0)

    # 3. Measurement (to the default classical register 'c')
    qc.measure(0, 0)
    
    return qc

def run_experiment(bomb_status, shots=1000):
    """Runs the circuit using the StatevectorSampler and prints the results."""
    qc = create_bomb_tester_circuit(bomb_status)
    
    # Run the circuit using the Sampler primitive
    job = sampler.run([qc], shots=shots)
    result = job.result()
    
    # FIX: Access the counts. 
    # result[0].data is a DataBin. The counts for the single classical register 
    # (named 'c' by default) is accessed as an attribute, and the data is a BitArray 
    # which has the .get_counts() method.
    counts = result[0].data.c.get_counts()
    
    # Calculate probabilities
    p_0 = counts.get('0', 0) / shots * 100
    p_1 = counts.get('1', 0) / shots * 100
    
    print(f"--- Experiment with a {bomb_status.upper()} BOMB ---")
    print(qc.draw(output='text', fold=-1))
    print(f"\nMeasurement Counts (Shots: {shots}): {counts}")
    print(f"Probability of measuring |0⟩ (Safe/Success): {p_0:.1f}%")
    print(f"Probability of measuring |1⟩ (Explosion/Failure): {p_1:.1f}%")

    # Interpretation
    if bomb_status == "dud":
        # Theory: 100% |0>
        print("OUTCOME: ✅ **Dud Detected (SAFE)**. All photons recombined on the $|0\rangle$ path.")
    elif bomb_status == "live":
        # Theory: 50% |0> (Success), 50% |1> (Explosion)
        if p_0 > 0:
            print(f"OUTCOME: 🌟 **Success (INTERACTION-FREE)**. Measurement '|0⟩' occurred. This proves the bomb is **LIVE** without triggering it! (Probability ~{p_0:.1f}%)")
        if p_1 > 0:
            print("OUTCOME: 💥 **Explosion/Failure** also occurred on this run. Measurement '|1⟩' means the photon took the upper path and exploded the bomb.")


# --- Run the Scenarios ---
print("\n"+"#"*70)
print("### SCENARIO 1: The BOMB IS A DUD ###")
run_experiment("dud", shots=1000)

print("\n"+"#"*70)
print("### SCENARIO 2: The BOMB IS LIVE ###")
run_experiment("live", shots=1000)
print("\n"+"#"*70)


######################################################################
### SCENARIO 1: The BOMB IS A DUD ###
--- Experiment with a DUD BOMB ---
     ┌───┐┌───┐┌───┐┌─┐
  q: ┤ H ├┤ I ├┤ H ├┤M├
     └───┘└───┘└───┘└╥┘
c: 1/════════════════╩═
                     0 

Measurement Counts (Shots: 1000): {'0': 1000}
Probability of measuring |0⟩ (Safe/Success): 100.0%
Probability of measuring |1⟩ (Explosion/Failure): 0.0%
angle$ path.*Dud Detected (SAFE)**. All photons recombined on the $|0

######################################################################
### SCENARIO 2: The BOMB IS LIVE ###
--- Experiment with a LIVE BOMB ---
     ┌───┐┌───┐┌───┐┌─┐
  q: ┤ H ├┤ Z ├┤ H ├┤M├
     └───┘└───┘└───┘└╥┘
c: 1/════════════════╩═
                     0 

Measurement Counts (Shots: 1000): {'1': 1000}
Probability of measuring |0⟩ (Safe/Success): 0.0%
Probability of measuring |1⟩ (Explosion/Failure): 100.0%
OUTCOME: 💥 **Explosion/Failure** also occurred on this run. Measurement '|1⟩' means the phot

In [28]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np

# Use the AerSimulator directly for the measurement process
simulator = AerSimulator(method='statevector')

def run_ev_experiment_correct(bomb_status, shots=1000):
    """
    Runs the corrected Elitzur-Vaidman experiment using conditional logic 
    to model the probabilistic explosion/collapse.
    """
    if bomb_status == "dud":
        # 1. DUD SCENARIO: The standard Mach-Zehnder interference
        qc = QuantumCircuit(1, 1, name="Dud_Test")
        qc.h(0)
        qc.id(0) # Dud (Identity)
        qc.h(0)
        qc.measure(0, 0)
        
        # Expected ideal result: 100% |0>
        
    elif bomb_status == "live":
        # 2. LIVE BOMB SCENARIO: The core probabilistic test
        # We must measure the path *before* the second H gate to model the explosion/collapse.
        qc = QuantumCircuit(1, 1, name="Live_Test_Probabilistic")
        qc.h(0)
        
        # This measurement models the bomb's interaction (explosion/collapse)
        # If the photon is in |1> (Upper Path), the bomb explodes (50% chance).
        qc.measure(0, 0)
        
        # We only continue the interferometer if the photon was NOT in the bomb path (m=0)
        # However, modeling the *non-detection* requires a different simulator setup.
        # The best we can do with standard Qiskit is model the conditional interference:
        
        # If the measurement result is 0 (Lower Path), apply the second H gate
        # Otherwise (measurement result is 1, Upper Path), the bomb exploded, and the experiment ends (no measurement at C or D).
        # We cannot model the 'No photon detected' (explosion) cleanly as a bitstring outcome.
        
        # We will use the standard EV circuit but rely on classical post-processing 
        # based on the measurement *before* the second H gate to model the explosion.
        pass # We will use the probabilistic analysis instead of the single circuit

    else:
        raise ValueError("Invalid bomb status.")


    if bomb_status == "dud":
        # Run the standard circuit
        t_qc = transpile(qc, simulator)
        job = simulator.run(t_qc, shots=shots)
        counts = job.result().get_counts()
        
        p_0 = counts.get('0', 0) / shots * 100
        p_1 = counts.get('1', 0) / shots * 100
        
        print(f"--- Experiment with a {bomb_status.upper()} BOMB ---")
        print(qc.draw(output='text', fold=-1))
        print(f"\nMeasurement Counts (Shots: {shots}): {counts}")
        print(f"P(|0⟩, Detector C): {p_0:.1f}%")
        print(f"P(|1⟩, Detector D): {p_1:.1f}%")
        print("OUTCOME: ✅ **Dud Detected (SAFE)**. Interference works perfectly (100% on C).")
        
    elif bomb_status == "live":
        # Simulate the first H gate: |ψ⟩ = (|0> + |1>)/√2
        # This is where the path choice is made. We use the Sampler to draw the path.
        qc_path = QuantumCircuit(1, 1, name="Live_Bomb_Path_Choice")
        qc_path.h(0)
        qc_path.measure(0, 0)
        
        job = sampler.run([qc_path], shots=shots)
        result = job.result()
        counts_path = result[0].data.c.get_counts()
        
        # Post-process the path choice
        path_1_count = counts_path.get('1', 0)  # Bomb Path (|1>)
        path_0_count = counts_path.get('0', 0)  # Safe Path (|0>)

        # 1. Explosion/No Photon: Occurs when the photon takes the |1> path.
        explosion_count = path_1_count 
        
        # 2. Interaction-Free Branch: Occurs when the photon takes the |0> path (Safe).
        # This safe path then hits the second H gate and splits 50/50.
        
        # Simulate the final split of the safe path
        qc_split = QuantumCircuit(1, 1, name="Final_Split")
        qc_split.h(0) # Starts in |0>, H splits to (|0> + |1>)/√2
        qc_split.measure(0, 0)
        
        job_split = simulator.run(qc_split, shots=path_0_count) # Only run for the safe-path count
        counts_split = job_split.result().get_counts()
        
        detector_C_count = counts_split.get('0', 0)
        detector_D_count = counts_split.get('1', 0)

        # Final Probabilities
        P_explosion = explosion_count / shots * 100
        P_C = detector_C_count / shots * 100
        P_D = detector_D_count / shots * 100
        
        print(f"--- Experiment with a {bomb_status.upper()} BOMB (Simulated Collapse) ---")
        print(f"Total Shots: {shots}")
        print(f"Path Choice Counts: {counts_path}")
        print("-" * 35)
        print("FINAL RESULTS:")
        print(f"P(No Photon Detected / Explosion): {P_explosion:.1f}% (Expected ~50.0%)")
        print(f"P(Detector C / Interaction-Free Success): {P_C:.1f}% (Expected ~25.0%)")
        print(f"P(Detector D / Safe Path): {P_D:.1f}% (Expected ~25.0%)")
        print("OUTCOME: 🌟 **Success Rate**: The probability of interaction-free detection (Detector C) is approximately 25%.")


# --- Run the Corrected Scenarios ---
print("\n"+"#"*70)
print("### SCENARIO 1: The BOMB IS A DUD (Corrected Model) ###")
run_ev_experiment_correct("dud", shots=1000)

print("\n"+"#"*70)
print("### SCENARIO 2: The BOMB IS LIVE (Corrected Probabilistic Model) ###")
run_ev_experiment_correct("live", shots=1000)
print("\n"+"#"*70)


######################################################################
### SCENARIO 1: The BOMB IS A DUD (Corrected Model) ###
--- Experiment with a DUD BOMB ---
     ┌───┐┌───┐┌───┐┌─┐
  q: ┤ H ├┤ I ├┤ H ├┤M├
     └───┘└───┘└───┘└╥┘
c: 1/════════════════╩═
                     0 

Measurement Counts (Shots: 1000): {'0': 1000}
P(|0⟩, Detector C): 100.0%
P(|1⟩, Detector D): 0.0%
OUTCOME: ✅ **Dud Detected (SAFE)**. Interference works perfectly (100% on C).

######################################################################
### SCENARIO 2: The BOMB IS LIVE (Corrected Probabilistic Model) ###
--- Experiment with a LIVE BOMB (Simulated Collapse) ---
Total Shots: 1000
Path Choice Counts: {'1': 500, '0': 500}
-----------------------------------
FINAL RESULTS:
P(No Photon Detected / Explosion): 50.0% (Expected ~50.0%)
P(Detector C / Interaction-Free Success): 23.7% (Expected ~25.0%)
P(Detector D / Safe Path): 26.3% (Expected ~25.0%)
OUTCOME: 🌟 **Success Rate**: The probability of interact